<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Final Project: Silicon Valley Neighborhood Research</font></h1>

### Introduction
This project is to investigate Silicon Valley Neighborhood based on FourSquare API and clustering data science method

### Set up Environment

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
import csv
import time

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library

### Download the data from data.gov & Convert raw data into pd.DataFrame

In [2]:
url = 'https://data.sfgov.org/api/views/f9wk-m4qb/rows.csv?accessType=DOWNLOAD'
data = pd.read_csv(url)

### **Part 1 Reform DataFrame

In [3]:
# Need to switch 'the_geom' into [latitude, longitude]; create area center as in 'latitude' and 'longitude' columns
import re
def extractFloat(temp):
    output = re.findall(r"[-+]?\d*\.\d+|\d+", temp)
    output = [float(i) for i in output]
    return output
    
geo = data["the_geom"].str.split("MULTIPOLYGON", n = 1, expand = True)[1] 
latitude  = []
longitude = []
for j in range(len(geo)):
    lat  = []
    long = []
    lat  = extractFloat(geo[j])[1::2]
    long = extractFloat(geo[j])[0::2]
    latitude.append(np.array(lat).mean())
    longitude.append(np.array(long).mean())

data['latitude']  = latitude
data['longitude'] = longitude
data = data[data['PO_NAME'] != 'TRAVIS AFB']
city_ls           = data.PO_NAME.unique().tolist()
print("There are {} unique cities found in Greater San Francisco Bay Area.".format(len(data.PO_NAME.unique())))
data.head()

There are 96 unique cities found in Greater San Francisco Bay Area.


,PO_NAME,the_geom,ZIP,STATE,Area__,Length__,latitude,longitude
0,NAPA,MULTIPOLYGON (((-122.10329200180091 38.5132829...,94558,CA,1.231326e+10,995176.225313,38.410697,-122.298535
1,FAIRFIELD,MULTIPOLYGON (((-121.947475002335 38.301511000...,94533,CA,9.917861e+08,200772.556587,38.279137,-122.025948
2,DIXON,MULTIPOLYGON (((-121.65335500334429 38.3133870...,95620,CA,7.236950e+09,441860.201400,38.434585,-121.781528
3,SONOMA,MULTIPOLYGON (((-122.406843003057 38.155681999...,95476,CA,3.001414e+09,311318.546326,38.279666,-122.456603
4,NAPA,MULTIPOLYGON (((-122.29368500225117 38.1552379...,94559,CA,1.194302e+09,359104.646602,38.266265,-122.311414


In [4]:
# Area List by City Name
area_ls = data.groupby('PO_NAME').sum()['Area__']

### **Part 2 Access to Foursqaure database

In [25]:
# Input Foursquare ID/Secret/Version
CLIENT_ID = 'TJ0SPQOHXOF5CJGO2BUHJM2ZXXEJK3K1WQZZWG2Y23XZN025' # your Foursquare ID
CLIENT_SECRET = 'NQEXK1ULW5FOTFHEOROXCF4U1VELGW4N2GAD4IYNDET0VIKI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: TJ0SPQOHXOF5CJGO2BUHJM2ZXXEJK3K1WQZZWG2Y23XZN025
CLIENT_SECRET:NQEXK1ULW5FOTFHEOROXCF4U1VELGW4N2GAD4IYNDET0VIKI
Latitude and longitude values of first row "NAPA" are (38.41069666247486, -122.29853512315688).


In [ ]:
# Test Run by only using the first row input
# Get neighborhood Data
bay_latitude = data.loc[0, 'latitude'] # neighborhood latitude value
bay_longitude = data.loc[0, 'longitude'] # neighborhood longitude value

bay_name = data.loc[0, 'PO_NAME'] # neighborhood name

print('Latitude and longitude values of first row "{}" are ({}, {}).'.format(bay_name, 
                                                               bay_latitude, 
                                                               bay_longitude))

LIMIT  = 20 # limit of number of venues returned by Foursquare API
radius = 10000 # define radius
price  = 2

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&price={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    bay_latitude, 
    bay_longitude, 
    radius,
    price,
    LIMIT)
url # display URL
results = requests.get(url).json()
#results

In [26]:
results

{'meta': {'code': 429,
  'errorType': 'quota_exceeded',
  'errorDetail': 'Quota exceeded',
  'requestId': '5d31f032fd16bb002c9b3081'},
 'response': {}}

In [22]:
# Function that extracts the category of the venue
LIMIT        = 20
radius_range = 10000

def getNearbyVenues(names, latitudes, longitudes, radius=radius_range):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Location', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Category']
    
    return(nearby_venues)

In [23]:
bay_venues = getNearbyVenues(names=data['PO_NAME'],
                             latitudes=data['latitude'],
                             longitudes=data['longitude']
                            )

print("Bay Area Venues shape: {}".format(bay_venues.shape))
bay_venues.head(20)

NAPA


KeyError: 'groups'

In [24]:
data['latitude']

0      38.410697
1      38.279137
2      38.434585
3      38.279666
4      38.266265
5      38.233673
6      38.189667
8      38.182079
9      38.068329
10     38.118574
11     38.145878
12     38.109064
13     38.088696
14     38.130289
15     38.138542
16     38.101083
17     38.098477
18     38.036234
19     38.023162
20     38.047533
21     38.045259
22     38.150717
23     37.772961
24     38.014832
25     38.014814
26     37.997220
27     38.036397
28     37.976970
29     38.010752
30     38.009207
31     38.010804
32     37.739115
33     37.989599
34     37.984291
35     37.924400
36     37.795740
37     37.969969
38     37.978224
39     38.002368
40     38.007517
41     37.920481
42     37.992372
43     37.944263
44     37.983617
45     37.973860
46     37.974008
47     37.965534
48     37.967609
49     37.780152
50     37.954923
51     37.952034
52     37.949329
53     37.785804
54     37.901570
55     37.909287
56     37.942559
57     37.920491
58     37.937543
59     37.9426

In [ ]:
bay_venues.rename(columns={'City Latitude':'latitude','City Longitude':'longitude'}, inplace=True)
baydata = pd.merge(bay_venues, data[['ZIP','longitude']], on='longitude')
print("The shape of Bay Area Raw Data extracted through FourSqaure API is {}".format(baydata.shape))
print("There are {} unique neighborhoods according to zip-code and {} unique cities.".format(len(baydata.ZIP.unique()),len(baydata.Location.unique())))
baydata.head()

### Answer of Q1 (shape)

In [ ]:
df.shape

### **Part 2 Add Geographic Information to dataframe

In [ ]:
import io 
csv_url = "http://cocl.us/Geospatial_data"
s       = requests.get(csv_url).content
df_geo  = pd.read_csv(io.StringIO(s.decode('utf-8')))

In [ ]:
df_geo.rename(columns={'Postal Code':'Postcode'}, inplace=True)
df_geo

In [ ]:
# Merge the two dataframes
df_new = pd.merge(df, df_geo, on='Postcode')

In [ ]:
df_new

In [ ]:
toronto_data = df_new[df_new['Borough'].str.contains(r'Toronto')].reset_index(drop=True)
toronto_data.head()

#### Get the geo info for Toronto

In [ ]:
#!conda install -c conda-forge geopy --yes
#!conda install -c conda-forge folium=0.5.0 --yes

from geopy.geocoders import Nominatim

In [ ]:
address = 'Toronto'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

### **Part 3: Clustering Toronto Neighborhood

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
toronto_data.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)
#toronto_data

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine the clusters

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

#### Conclusions:
Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, here are the cluster name and its category:
- Cluster 0: Coffee, Food and General Stores
- Cluster 1: Trasnportation (Bus Line)
- Cluster 2: Pool
- Cluster 3: Park
- Cluster 4: Other Restaurant

In [3]:
cl = ['Chinese Restaurant','Japanese Restaurant','Bus Station','Italian Restaurant','Korean Restaurant','Train Station']
df=pd.DataFrame(columns= cl)
clls=df.columns.tolist()
clls

['Chinese Restaurant',
 'Japanese Restaurant',
 'Bus Station',
 'Italian Restaurant',
 'Korean Restaurant',
 'Train Station']

In [6]:
mask = [i.find('Restaurant')>=0 for i in clls]
from itertools import compress
df[list(compress(clls,mask))]

,Chinese Restaurant,Japanese Restaurant,Italian Restaurant,Korean Restaurant
